# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

import sys
sys.path.append('../')

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

from IPython.display import Markdown

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 56 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.03041/final_version.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.03108/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'rv_table' from 'tmp_2204.03108/rv_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/Users/fouesneau/Work/arxiver_on_deck_2/

✔ → 0:header
  ↳ 21423:\section{Introduction}
✔ → 21423:\section{Introduction}
  ↳ 27679:\section{The instrument and data reduction}
✔ → 27679:\section{The instrument and data reduction}
  ↳ 39829:\section{Results}
✔ → 39829:\section{Results}
  ↳ 68451:\section{Physical interpretation }
✔ → 68451:\section{Physical interpretation }
  ↳ 71010:\section{Discussion and Summary}
✔ → 71010:\section{Discussion and Summary}
  ↳ 73313:\section{Acknowledgments}
✔ → 73313:\section{Acknowledgments}
  ↳ 74183:end


/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure FigMELCdeadtime
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure FigHXMT_ME_lcfit4
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure FigMELeahyPower
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure FigBPLandPLandBLorentzfit
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure FigPLposteriorD

✔ → 0:header
  ↳ 8441:\section{Introduction}
✘ → 8441:\section{Introduction}
  ↳ 16096:\section{Quasar Sample and Composites} \label{sec:sample-composites}
✔ → 16096:\section{Quasar Sample and Composites} \label{sec:sample-composites}
  ↳ 37058:\section{Line Fitting and Metallicity Measurement} \label{sec:metallicity}
✔ → 37058:\section{Line Fitting and Metallicity Measurement} \label{sec:metallicity}
  ↳ 56601:\section{Results} \label{sec:results}
✔ → 56601:\section{Results} \label{sec:results}
  ↳ 77694:\section{Discussion} \label{sec:discussion}
✔ → 77694:\section{Discussion} \label{sec:discussion}
  ↳ 89134:\section{Conclusions} \label{sec:conclusion}
✔ → 89134:\section{Conclusions} \label{sec:conclusion}
  ↳ 95750:\section{Comparison of emission-line fitting methods} \label{appendix:line-fitting-compare}
✔ → 95750:\section{Comparison of emission-line fitting methods} \label{appendix:line-fitting-compare}
  ↳ 100507:\section{Additional Tables and Figures}
✘ → 100507:\section{Additi

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: LatexWarning: Found documentclass in tmp_2204.03423/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'lisa_numbers' from 'tmp_2204.03423/lisa_numbers.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: '3g_numbers' from 'tmp_2204.03423/3g_numbers.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/Users/fouesneau/Work/arxiver_on_deck_2/examples/../arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'numbers' from 'tmp_2204.03423/numbers.tex'

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02998-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02998) | **High-contrast, high-angular resolution view of the GJ 367 exoplanet  system**  |
|| Wolfgang Brandner, Per Calissendorff, <mark>Neige Frankel</mark>, Faustine Cantalloube |
|*Comments*| *9 pages, 8 figures, accepted by MNRAS*|
|**Abstract**| We search for additional companions in the GJ 367 exoplanet system, and aim at better constraining its age and evolutionary status. We analyse high contrast direct imaging observations obtained with HST/NICMOS, VLT/NACO, and VLT/SPHERE. We investigate and critically discuss conflicting age indicators based on theoretical isochrones and models for Galactic dynamics. A comparison of GAIA EDR3 parallax and photometric measurements with theoretical isochrones suggest a young age $\le$60 Myr for GJ 367. The star's Galactic kinematics exclude membership to any nearby young moving group or stellar stream. Its highly eccentric Galactic orbit, however, is atypical for a young star. Age estimates considering Galactic dynamical evolution are most consistent with an age of 1 to 8 Gyr. We find no evidence for a significant mid-infrared excess in the WISE bands, suggesting the absence of warm dust in the GJ 367 system. The direct imaging data provide significantly improved detection limits compared to previous studies. At 530 mas (5 au) separation, the SPHERE data achieve a 5 sigma contrast of $2.6 \times 10^{-6}$. The data exclude the presence of a stellar companion at projected separations $\ge$0.4 au. At projected separations $\ge$5 au we can exclude substellar companions with a mass $\ge$ 1.5 M$_{\rm Jup}$ for an age of 50 Myr, and $\ge$ 20 M$_{\rm Jup}$ for an age of 5 Gyr. By applying the stellar parameters corresponding to the 50 Myr isochrone, we derive a bulk density of $\rho_{\rm planet} = 6.2$ g/cm$^3$ for GJ 367b, which is 25% smaller than a previous estimate. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03253-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03253) | **Quasi-periodic oscillations of the X-ray burst from the magnetar SGR  J1935+2154 and associated with the fast radio burst FRB 200428**  |
|| Xiaobo Li, et al. -- incl., <mark>Shuang-Nan Zhang</mark>, <mark>Bing Zhang</mark>, <mark>Shu Zhang</mark>, <mark>Binbin Zhang</mark>, <mark>Peng Zhang</mark>, <mark>He Gao</mark>, <mark>Xiaohua Liang</mark>, <mark>Hexin Liu</mark>, <mark>Xiang Ma</mark>, <mark>Liang Sun</mark>, <mark>Lian Tao</mark> |
|*Comments*| *17 pages, 13 figures, accepted for publication by ApJ. arXiv admin note: text overlap with arXiv:1212.1011 by other authors. text overlap with arXiv:1212.1011 by other authors*|
|**Abstract**| The origin(s) and mechanism(s) of fast radio bursts (FRBs), which are short radio pulses from cosmological distances, have remained a major puzzle since their discovery. We report a strong Quasi-Periodic Oscillation(QPO) of 40 Hz in the X-ray burst from the magnetar SGR J1935+2154 and associated with FRB 200428, significantly detected with the Hard X-ray Modulation Telescope (Insight-HXMT) and also hinted by the Konus-Wind data. QPOs from magnetar bursts have only been rarely detected; our 3.4 sigma (p-value is 2.9e-4) detection of the QPO reported here reveals the strongest QPO signal observed from magnetars (except in some very rare giant flares), making this X-ray burst unique among magnetar bursts. The two X-ray spikes coinciding with the two FRB pulses are also among the peaks of the QPO. Our results suggest that at least some FRBs are related to strong oscillation processes of neutron stars. We also show that we may overestimate the significance of the QPO signal and underestimate the errors of QPO parameters if QPO exists only in a fraction of the time series of a X-ray burst which we use to calculate the Leahy-normalized periodogram. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03269-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03269) | **Free-Floating Planets, the Einstein Desert, and 'Oumuamua**  |
|| <mark>Andrew Gould</mark>, et al. -- incl., <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Comments*| *20 pages, 9 Figures, submitted to JKAS*|
|**Abstract**| We complete the survey for finite-source/point-lens (FSPL) giant-source events in 2016-2019 KMTNet microlensing data. The 30 FSPL events show a clear gap in Einstein radius, $9\,\mu{\rm as}<\theta_{\rm E} <26\,\mu{\rm as}$, which is consistent with the gap in Einstein timescales near $t_{\rm E}\sim 0.5\,$days found by Mroz et al. (2017) in an independent sample of point-source/point-lens (PSPL) events. We demonstrate that the two surveys are consistent. We estimate that the 4 events below this gap are due to a power-law distribution of free-floating planet candidates (FFPs) $dN_{\rm FFP}/d\log M = (0.4\pm 0.2)\,(M/38 M_\oplus)^{-p}$/star, with $0.9\lesssim p \lesssim 1.2$. There are substantially more FFPs than known bound planets, implying that the bound planet power-law index $\gamma=0.6$ is likely shaped by the ejection process at least as much as by formation. The mass density per decade of FFPs in the Solar neighborhood is of the same order as that of 'Oumuamua-like objects. In particular, if we assume that 'Oumuamua is part of the same process that ejected the FFPs to very wide or unbound orbits, the power-law index is $p=0.92\pm 0.06$. If the Solar System's endowment of Neptune-mass objects in Neptune-like orbits is typical, which is consistent with the results of Poleski et al. (2021), then these could account for a substantial fraction of the FFPs in the Neptune-mass range. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03335-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03335) | **Chemical Abundance of z~6 Quasar Broad-Line Regions in the XQR-30 Sample**  |
|| Samuel Lai, et al. -- incl., <mark>Sarah E.I. Bosman</mark>, <mark>Jan-Torge Schindler</mark>, <mark>Fabian Walter</mark> |
|*Comments*| *MNRAS accepted, 24 pages, 17 figures, 4 tables*|
|**Abstract**| The elemental abundances in the broad-line regions of high-redshift quasars trace the chemical evolution in the nuclear regions of massive galaxies in the early universe. In this work, we study metallicity-sensitive broad emission-line flux ratios in rest-frame UV spectra of 25 high-redshift (5.8 < z < 7.5) quasars observed with the VLT/X-shooter and Gemini/GNIRS instruments, ranging over $\log(M_{\rm{BH}}/M_{\odot})= 8.4-9.8$ in black hole mass and $\log(L_{\rm{bol}}/\rm{erg\, s}^{-1})= 46.7-47.7$ in bolometric luminosity. We fit individual spectra and composites generated by binning across quasar properties: bolometric luminosity, black hole mass, and blueshift of the \civ\, line, finding no redshift evolution in the emission-line ratios by comparing our high-redshift quasars to lower-redshift (2.0 < z < 5.0) results presented in the literature. Using Cloudy-based locally optimally-emitting cloud photoionisation model relations between metallicity and emission-line flux ratios, we find the observable properties of the broad emission lines to be consistent with emission from gas clouds with metallicity that are at least 2-4 times solar. Our high-redshift measurements also confirm that the blueshift of the CIV emission line is correlated with its equivalent width, which influences line ratios normalised against CIV. When accounting for the CIV blueshift, we find that the rest-frame UV emission-line flux ratios do not correlate appreciably with the black hole mass or bolometric luminosity. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03327-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03327) | **Constraints on TESS albedos for five hot Jupiters**  |
|| Martin Blažek, et al. -- incl., <mark>Henri M. J. Boffin</mark>, <mark>Sergio Hoyer</mark> |
|*Comments*| *21 pages, 7 figures*|
|**Abstract**| Photometric observations of occultations of transiting exoplanets can place important constraints on the thermal emission and albedos of their atmospheres. We analyse photometric measurements and derive geometric albedo ($A_\mathrm{g}$) constraints for five hot Jupiters observed with TESS in the optical: WASP-18 b, WASP-36 b, WASP-43 b, WASP-50 b and WASP-51 b. For WASP-43 b, our results are complemented by a VLT/HAWK-I observation in the near-infrared at $2.09~\mu$m. We derive the first geometric albedo constraints for WASP-50 b and WASP-51 b: $A_\mathrm{g}<0.445$ and $A_\mathrm{g}<0.368$, respectively. We find that WASP-43 b and WASP-18 b are both consistent with low geometric albedos ($A_\mathrm{g}<0.16$) even though they lie at opposite ends of the hot Jupiter temperature range with equilibrium temperatures of $\sim1400$ K and $\sim2500$ K, respectively. We report self-consistent atmospheric models which explain broadband observations for both planets from TESS, \HST, \Spitzer and VLT/HAWK-I. We find that the data of both hot Jupiters can be explained by thermal emission alone and inefficient day-night energy redistribution. The data do not require optical scattering from clouds/hazes, consistent with the low geometric albedos observed. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03358-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03358) | **Ultracool dwarfs identified using spectra in LAMOST DR7**  |
|| You-Fen Wang, et al. -- incl., <mark>Shuo Zhang</mark>, <mark>Zhong-Rui Bai</mark> |
|*Comments*| *13 pages, 11 figures, published in A&A, Volume 660. A38 (2022)*|
|**Abstract**| In this work, we identify 734 ultracool dwarfs with a spectral type of M6 or later, including one L0. Of this sample, 625 were studied spectroscopically for the first time. All of these ultracool dwarfs are within 360~pc, with a \textit{Gaia} G magnitude brighter than ~19.2 mag. By studying the spectra and checking their stellar parameters (Teff, logg, and [FeH] derived with the LAMOST pipeline, we found their cool red nature and their metallicity to be consistent with the nature of Galactic thin-disk objects. Furthermore, 77 of them show lithium absorption lines at 6708A, further indicating their young ages and substellar nature. Kinematics obtained through LAMOST radial velocities, along with the proper motion and parallax data from Gaia EDR3, also suggest that the majority of our targets are thin-disk objects. Kinematic ages were estimated through the relationship between the velocity dispersion and the average age for a certain population. Moreover, we identified 35 binaries, with 6 of them reported as binaries for the first time. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03423-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03423) | **The last three years: multiband gravitational-wave observations of  stellar-mass binary black holes**  |
|| <mark>Antoine Klein</mark>, et al. -- incl., <mark>Natalie Williams</mark> |
|*Comments*| *21 pages, 13 figures (including supplementary material)*|
|**Abstract**| Understanding the formation and evolution of the stellar-mass binary black holes discovered by LIGO and Virgo is a challenge that spans many areas of astrophysics, from stellar evolution, dynamics and accretion disks, to possible exotic early universe processes. Over the final years of their lives, stellar-mass binaries radiate gravitational waves that are first observable by space-based detectors (such as LISA) and then ground-based instruments (such as LIGO, Virgo and the next generation observatories Cosmic Explorer and the Einstein Telescope). Using state-of-the-art waveform models and parameter-estimation pipelines for both ground- and space-based observations, we show that (the expected handful of) these multiband observations will allow at least percent-level measurements of all 17 parameters that describe the binary, the possible identification of a likely host galaxy, and the forewarning of the merger days in advance allowing telescopes at multiple wavelengths to search for any electromagnetic signature associated to it. Multiband sources will therefore be a gold mine for astrophysics, but we also show that they are less useful as laboratories for fundamental tests of general relativity than has been previously suggested. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03457-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03457) | **Validation and interpretation of three-dimensional configuration of a  magnetic cloud flux rope**  |
|| Qiang Hu, et al. -- incl., <mark>Wen He</mark> |
|*Comments*| *Submitted to ApJ*|
|**Abstract**| One "strong" magnetic cloud (MC) with the magnetic field magnitude reaching $\sim$ 40 nT at 1 au during 2012 June 16-17 is examined in association with a pre-existing magnetic flux rope (MFR) identified on the Sun. The MC is characterized by a quasi-three dimensional (3D) flux rope model based on in situ measurements from the Wind spacecraft. The magnetic flux contents and other parameters are quantified. In addition, a correlative study with the corresponding measurements of the same structure crossed by the Venus Express (VEX) spacecraft at a heliocentric distance 0.7 au and with an angular separation $\sim 6^\circ$ in longitude is performed to validate the MC modeling results. The spatial variation between the Wind and VEX magnetic field measurements is attributed to the 3D configuration of the structure as featured by a knotted bundle of flux. The comparison of the magnetic flux contents between the MC and the source region on the Sun indicates that the 3D reconnection process accompanying an M1.9 flare may correspond to the magnetic reconnection between the field lines of the pre-existing MFR rooted in the opposite polarity footpoints. Such a process reduces the amount of the axial magnetic flux in the erupted flux rope, by approximately 50\%, in this case. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03108-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03108) | **A close-in puffy Neptune with hidden friends: The enigma of TOI 620**  |
|| Michael A. Reefe, et al. -- incl., <mark>Fred Hearty</mark> |
|*Comments*| *64 pages, 34 figures, 22 tables. Accepted for publication in the Astronomical Journal*|
|**Abstract**| We present the validation of a transiting low-density exoplanet orbiting the M2.5 dwarf TOI 620 discovered by the NASA TESS mission. We utilize photometric data from both TESS and ground-based follow-up observations to validate the ephemerides of the 5.09-day transiting signal and vet false positive scenarios. High-contrast imaging data are used to resolve the stellar host and exclude stellar companions at separations $\gtrsim 0.2''$. We obtain follow-up spectroscopy and corresponding precise radial velocities (RVs) with multiple PRV spectrographs to confirm the planetary nature of the transiting exoplanet. We calculate a 5$\sigma$ upper limit of $M_P < 7.1$ M$_\oplus$ and $\rho_P < 0.74$ g cm$^{-3}$, and we identify a non-transiting 17.7-day candidate. We also find evidence for a substellar (1-20 M$_{\rm J}$) companion with a projected separation $\lesssim 20$ au from a combined analysis of Gaia, AO imaging, and RVs. With the discovery of this outer companion, we carry out a detailed exploration of the possibilities that TOI 620 b might instead be a circum-secondary planet or a pair of eclipsing binary stars orbiting the host in a hierarchical triple system. We find, under scrutiny, that we can exclude both of these scenarios from the multi-wavelength transit photometry, thus validating TOI 620 b as a low-density exoplanet transiting the central star in this system. The low density of TOI 620 b makes it one of the most amenable exoplanets for atmospheric characterization, such as with JWST and Ariel, validated or confirmed by the TESS mission to date. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", 'exports')

exported in  exports/2204.02998.md
    + exports/tmp_2204.02998/./figures/gj367_nic2_naco.png
    + exports/tmp_2204.02998/./figures/Mdwarfs_iso_GAIA.png
    + exports/tmp_2204.02998/./figures/gj367_2mass_iso.png
    + exports/tmp_2204.02998/./figures/gj367_radial_contrast.png
exported in  exports/2204.03253.md
    + exports/tmp_2204.03253/./FigHXMT_ME_lcfit4.png
    + exports/tmp_2204.03253/./sin_nois_lc.png
    + exports/tmp_2204.03253/./Figall_LRT.png
exported in  exports/2204.03269.md
    + exports/tmp_2204.03269/./cmdall.png
    + exports/tmp_2204.03269/./lb.png
    + exports/tmp_2204.03269/./cum3.png
    + exports/tmp_2204.03269/./z.png
exported in  exports/2204.03335.md
    + exports/tmp_2204.03335/./figs/line_ratio_blueshift.png
    + exports/tmp_2204.03335/./figs/CIV_Blueshift_EW.png
    + exports/tmp_2204.03335/./figs/line_ratio_Lbol.png
    + exports/tmp_2204.03335/./figs/line_ratio_Mbh.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# High-contrast, high-angular resolution view of the GJ 367 exoplanet system

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2204.02998-b31b1b.svg)](https://arxiv.org/abs/2204.02998) _9 pages, 8 figures, accepted by MNRAS_

</div>
<div id="authors">

Wolfgang Brandner, Per Calissendorff, <mark><mark>Neige Frankel</mark></mark>, Faustine Cantalloube

</div>
<div id="abstract">

**Abstract:** We search for additional companions in the GJ 367 exoplanet system, and aim at better constraining its age and evolutionary status.We analyse high contrast direct imaging observations obtained with HST/NICMOS, VLT/NACO, and VLT/SPHERE. We investigate and critically discuss conflicting age indicators based on theoretical isochrones and models for Galactic dynamics.A comparison of GAIA EDR3 parallax and photometric measurements with theoretical isochrones suggest a young age$\le$60 Myr for GJ 367. The star's Galactic kinematics exclude membership to any nearby young moving group or stellar stream. Its highly eccentric Galactic orbit, however, is atypical for a young star. Age estimates considering Galactic dynamical evolution are most consistent with an age of 1 to 8 Gyr. We find no evidence for a significant mid-infrared excess in the WISE bands, suggesting the absence of warm dust in the GJ 367 system.The direct imaging data provide significantly improved detection limits compared to previous studies. At 530 mas (5 au) separation, the SPHERE data achieve a 5 sigma contrast of$2.6 \times 10^{-6}$. The data exclude the presence of a stellar companion at projected separations$\ge$0.4 au. At projected separations$\ge$5 au we can exclude substellar companions with a mass$\ge$1.5 M$_{\rm Jup}$for an age of 50 Myr, and$\ge$20 M$_{\rm Jup}$for an age of 5 Gyr.By applying the stellar parameters corresponding to the 50 Myr isochrone, we derive a bulk density of$\rho_{\rm planet} = 6.2$g/cm$^3$for GJ 367 b, which is 25\%smaller than a previous estimate.

</div>

<div id="div_fig1">

<img src="tmp_2204.02998/./figures/gj367_nic2_naco.png" alt="Fig5" width="100%"/>

**Figure 5. -** Left: HST NIC2 image in F110W centered on GJ 367, with five companion candidates (cc) visible (black circles). The location of residuals originating in NIC2's coronagraphic mask is indicated (white circle). Right: VLT/NACO image in NB175 centered on GJ 367. The lower left insert shows a $4\times$ zoom-in on an individual 10s frame depicting a side-lope resulting from the low wind effect. The insert on the lower right shows a $4\times$ zoom-in of the coadded PSF. (*GJ367_NIC2_NACO*)

</div>
<div id="div_fig2">

<img src="tmp_2204.02998/./figures/Mdwarfs_iso_GAIA.png" alt="Fig7.1" width="50%"/><img src="tmp_2204.02998/./figures/gj367_2mass_iso.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Colour-magnitude diagrams for GJ 367 compared of theoretical models by \cite{Baraffe2015}. The plot on the left shows isochrones in the GAIA photometric systems, with GJ 367 (red symbol) being located below the 50 Myr and above the 80 Myr isochrone. The plot on the right illustrates that isochrones transformed to 2MASS bands do not provide any meaningful constraint on GJ 367's age. The 1 $\sigma$ error bars in both plots are based on the uncertainties in photometry and parallax. (*GJ367_age_mag*)

</div>
<div id="div_fig3">

<img src="tmp_2204.02998/./figures/gj367_radial_contrast.png" alt="Fig1" width="100%"/>

**Figure 1. -** Radial 5 $\sigma$ contrast achieved with NIC2 in the F180M filter (dash-dotted line, \cite{Dieterich2012}), and SADI processed SPHERE/IRDIS in the H2/H3-filters (red dashed line) plotted over the projected separation. (*contrast_h2*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\insight}{\textit{Insight}-HXMT}$
$\newcommand{\sgr}{\mbox{SGR~J1935+2154~}}$
$\newcommand{\sgrnos}{\mbox{SGR~J1935+2154}}$
$\newcommand{\modify}[1]{\textcolor{blue}{\textbf{#1}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\insight$}{\textit{Insight}-HXMT}$
$\newcommand{$\sgr$}{\mbox{SGR~J1935+2154~}}$
$\newcommand{$\sgr$nos}{\mbox{SGR~J1935+2154}}$
$\newcommand{$\modify$}[1]{\textcolor{blue}{\textbf{#1}}}$</div>



<div id="title">

# Quasi-periodic oscillations of the X-ray burst from the magnetar SGR J1935+2154 and associated with the fast radio burst FRB 200428

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2204.03253-b31b1b.svg)](https://arxiv.org/abs/2204.03253) _17 pages, 13 figures, accepted for publication by ApJ. arXiv admin note: text overlap with arXiv:1212.1011 by other authors. text overlap with arXiv:1212.1011 by other authors_

</div>
<div id="authors">

Xiaobo Li, et al. -- incl., <mark><mark>Shuang-Nan Zhang</mark></mark>, <mark><mark>Bing Zhang</mark></mark>, <mark><mark>Shu Zhang</mark></mark>, <mark><mark>Binbin Zhang</mark></mark>, <mark><mark>Peng Zhang</mark></mark>, <mark><mark>He Gao</mark></mark>, <mark><mark>Xiaohua Liang</mark></mark>, <mark><mark>Hexin Liu</mark></mark>, <mark><mark>Xiang Ma</mark></mark>, <mark><mark>Liang Sun</mark></mark>, <mark><mark>Lian Tao</mark></mark>

</div>
<div id="abstract">

**Abstract:** The origin(s) and mechanism(s) of fast radio bursts (FRBs), which are short radio pulses from cosmological distances, have remained a major puzzle since their discovery. We report a strong Quasi-Periodic Oscillation (QPO) of$\sim$40 Hz in the X-ray burst from the magnetar$\sgr$and associated with FRB 200428, significantly detected with the Hard X-ray Modulation Telescope (\textit{Insight}-HXMT) and also hinted by the Konus-Wind data. QPOs from magnetar bursts have only been rarely detected; our 3.4$\sigma$(p-value is 2.9e-4) detection of the QPO reported here reveals the strongest QPO signal observed from magnetars (except in some very rare giant flares), making this X-ray burst unique among magnetar bursts. The two X-ray spikes coinciding with the two FRB pulses are also among the peaks of the QPO. Our results suggest that at least some FRBs are related to strong oscillation processes of neutron stars.We also show that we may overestimate the significance of the QPO signal and underestimate the errors of QPO parameters if QPO exists only in a fraction of the time series of a X-ray burst which we use to calculate the Leahy-normalized periodogram.

</div>

<div id="div_fig1">

<img src="tmp_2204.03253/./FigHXMT_ME_lcfit4.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
Light curve of the X-ray burst in 18--50 keV obtained by $\insight$  /ME with time resolution of 5 ms. The blue line is the light curve after dead time correction. The green line represents the fit result of the burst profile with three Gaussian functions as given in eq. (\ref{eq0}). The red line is the fitted oscillation curve with eq. (\ref{eq2}). The two dashed green lines show the lower and upper envelopes of the amplitude variation of the QPOs described by eq. (\ref{eq_en}). The two vertical dashed lines denote the times of the two radio pulses of FRB 200428. The inset is the light curve around the burst peak with the solid green line subtracted.
\label{fig_MElconly} (*fig_MElconly*)

</div>
<div id="div_fig2">

<img src="tmp_2204.03253/./sin_nois_lc.png" alt="Fig11" width="100%"/>

**Figure 11. -** 
Simulated light curves with sinusoidal signals and only white noise. {\bf Upper panel (a):} simulated light curves with 3 seconds of sinusoidal signals of amplitudes 8 and 24, respectively. {\bf Lower panel (b):} simulated light curves with only white noise with Poisson expectation value of 24 in each time bin.
\label{fig_sin_lc} (*fig_sin_lc*)

</div>
<div id="div_fig3">

<img src="tmp_2204.03253/./Figall_LRT.png" alt="Fig8" width="100%"/>

**Figure 8. -** 
Distributions of likelihood ratio test for 150000, 2000000 and 2000000 simulations of the null hypothesis for 0.32 s (black), 0.64 s (red) and 1.28 s (blue), respectively. The corresponding $H_0$ model is power law, broken power law and broken power law, respectively. The observed values of LRT are indicated as vertical lines. The ratios of the tail area for 0.32 s and 0.64 s are 2.9e-4 and 1e-6, both are indicating that a more complex model including additional Lorentz model may be more appropriate in modeling the periodogram. As for the 1.28 s duration, the tail area is zero and the p-value is an upper limit.
\label{fig_allLRT} (*fig_allLRT*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{\msun}{{\rm  M_\odot}}$
$\newcommand{\bdv}[1]{\mbox{\boldmath#1}}$
$\newcommand{\bd}[1]{{\rm #1}}$
$\newcommand{\year}{2022}$
$\newcommand{\month}{April}$
$\newcommand{\volume}{00}$
$\newcommand{\issue}{0}$
$\newcommand{\beginpage}{1}$
$\newcommand{\endpage}{99}$
$\newcommand{\received}{---}$
$\newcommand{\accepted}{---}$
$\newcommand{\}{corrauthor}$
$\newcommand{\}{runningauthor}$
$\newcommand{\}{runningtitle}$
$\newcommand{\}{keywords}$
$\newcommand{\}{abstracttext}$
$\newcommand{\au}{{\rm AU}}$
$\newcommand{\bx}{{\bf x}}$
$\newcommand{\bv}{{\bf v}}$
$\newcommand{\bQ}{{\bf Q}}$
$\newcommand{\vega}{{\rm vega}}$
$\newcommand{\cam}{{\rm cam}}$
$\newcommand{\snr}{{\rm SNR}}$
$\newcommand{\sinc}{{\rm sinc}}$
$\newcommand{\kms}{{\rm km} {\rm s}^{-1}}$
$\newcommand{\masyr}{{\rm mas} {\rm yr}^{-1}}$
$\newcommand{\muas}{{\mu\rm as}}$
$\newcommand{\mas}{{\rm mas}}$
$\newcommand{\kpc}{{\rm kpc}}$
$\newcommand{\var}{{\rm var}}$
$\newcommand{\pc}{{\rm pc}}$
$\newcommand{\orb}{{\rm orb}}$
$\newcommand{\obs}{{\rm obs}}$
$\newcommand{\rot}{{\rm rot}}$
$\newcommand{\max}{{\rm max}}$
$\newcommand{\rel}{{\rm rel}}$
$\newcommand{\ast}{{\rm ast}}$
$\newcommand{\eff}{{\rm eff}}$
$\newcommand{\sat}{{\rm sat}}$
$\newcommand{\hel}{{\rm hel}}$
$\newcommand{\geo}{{\rm geo}}$
$\newcommand{\single}{{\rm single}}$
$\newcommand{\phot}{{\rm phot}}$
$\newcommand{\unsat}{{\rm unsat}}$
$\newcommand{\mahn}{{\rm mahn}}$
$\newcommand{\feh}{{\rm[Fe/H]}}$
$\newcommand{\lim}{{\rm lim}}$
$\newcommand{\e}{{\rm E}}$
$\newcommand{\bpi}{{\bdv\pi}}$
$\newcommand{\bmu}{{\bdv\mu}}$
$\newcommand{\bnu}{{\bdv\nu}}$
$\newcommand{\btheta}{{\bdv\theta}}$
$\newcommand{\balpha}{{\bdv\alpha}}$
$\newcommand{\bDelta}{{\bdv\Delta}}$
$\newcommand{\bp}{{\bf p}}$
$\newcommand{\bD}{{\bf D}}$
$\newcommand{\bP}{{\bf P}}$
$\newcommand{\pix}{{\rm pixel}}$
$\newcommand{\hbn}{{\hfil\break\noindent}}$
$\newcommand{\la}{{\lesssim}}$
$\newcommand{\ga}{{\gtrsim}}$
$\newcommand{\apj}{{ApJ}}$
$\newcommand{\aj}{{AJ}}$
$\newcommand{\apjl}{{ApJL}}$
$\newcommand{\apjs}{{ApJS}}$
$\newcommand{\aap}{{A\&A}}$
$\newcommand{\pasp}{{PASP}}$
$\newcommand{\mnras}{{MNRAS}}$
$\newcommand{\araa}{{ARA\&A}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand$
$\newcommand{$\msun$}{{\rm  M_\odot}}$
$\newcommand{$\bdv$}[1]{\mbox{\boldmath#1}}$
$\newcommand{$\bd$}[1]{{\rm #1}}$
$\newcommand{$\year$}{2022}$
$\newcommand{$\month$}{April}$
$\newcommand{$\volume$}{00}$
$\newcommand{$\issue$}{0}$
$\newcommand{$\beginpage$}{1}$
$\newcommand{$\endpage$}{99}$
$\newcommand{$\received$}{---}$
$\newcommand{$\accepted$}{---}$
$\newcommand{\}{corrauthor}$
$\newcommand{\}{runningauthor}$
$\newcommand{\}{runningtitle}$
$\newcommand{\}{keywords}$
$\newcommand{\}{abstracttext}$
$\newcommand{$\au$}{{\rm AU}}$
$\newcommand{$\bx$}{{\bf x}}$
$\newcommand{$\bv$}{{\bf v}}$
$\newcommand{$\bQ$}{{\bf Q}}$
$\newcommand{$\vega$}{{\rm vega}}$
$\newcommand{$\cam$}{{\rm cam}}$
$\newcommand{$\snr$}{{\rm SNR}}$
$\newcommand{$\sinc$}{{\rm sinc}}$
$\newcommand{$\kms$}{{\rm km} {\rm s}^{-1}}$
$\newcommand{$\masyr$}{{\rm mas} {\rm yr}^{-1}}$
$\newcommand{$\muas$}{{\mu\rm as}}$
$\newcommand{$\mas$}{{\rm mas}}$
$\newcommand{$\kpc$}{{\rm kpc}}$
$\newcommand{$\var$}{{\rm var}}$
$\newcommand{$\pc$}{{\rm pc}}$
$\newcommand{$\orb$}{{\rm orb}}$
$\newcommand{$\obs$}{{\rm obs}}$
$\newcommand{$\rot$}{{\rm rot}}$
$\newcommand{$\max$}{{\rm max}}$
$\newcommand{$\rel$}{{\rm rel}}$
$\newcommand{$\ast$}{{\rm ast}}$
$\newcommand{$\eff$}{{\rm eff}}$
$\newcommand{$\sat$}{{\rm sat}}$
$\newcommand{$\hel$}{{\rm hel}}$
$\newcommand{$\geo$}{{\rm geo}}$
$\newcommand{$\single$}{{\rm single}}$
$\newcommand{$\phot$}{{\rm phot}}$
$\newcommand{$\unsat$}{{\rm unsat}}$
$\newcommand{$\mahn$}{{\rm mahn}}$
$\newcommand{$\feh$}{{\rm[Fe/H]}}$
$\newcommand{$\lim$}{{\rm lim}}$
$\newcommand{$\e$}{{\rm E}}$
$\newcommand{$\bpi$}{{$\bdv$\pi}}$
$\newcommand{$\bmu$}{{$\bdv$\mu}}$
$\newcommand{$\bnu$}{{$\bdv$\nu}}$
$\newcommand{$\btheta$}{{$\bdv$\theta}}$
$\newcommand{$\balpha$}{{$\bdv$\alpha}}$
$\newcommand{$\bDelta$}{{$\bdv$\Delta}}$
$\newcommand{$\bp$}{{\bf p}}$
$\newcommand{$\bD$}{{\bf D}}$
$\newcommand{$\bP$}{{\bf P}}$
$\newcommand{$\pix$}{{\rm pixel}}$
$\newcommand{$\hbn$}{{\hfil\break\noindent}}$
$\newcommand{$\la$}{{\lesssim}}$
$\newcommand{$\ga$}{{\gtrsim}}$
$\newcommand{$\apj$}{{ApJ}}$
$\newcommand{$\aj$}{{AJ}}$
$\newcommand{$\apj$l}{{ApJL}}$
$\newcommand{$\apj$s}{{ApJS}}$
$\newcommand{$\aap$}{{A\&A}}$
$\newcommand{$\pasp$}{{PASP}}$
$\newcommand{$\mnras$}{{MNRAS}}$
$\newcommand{$\araa$}{{ARA\&A}}$</div>



<div id="title">

# Free-Floating Planets, the Einstein Desert, and 'Oumuamua

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2204.03269-b31b1b.svg)](https://arxiv.org/abs/2204.03269) _20 pages, 9 Figures, submitted to JKAS_

</div>
<div id="authors">

<mark>Andrew Gould</mark>, et al. -- incl., <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark>

</div>
<div id="abstract">

**Abstract:** We complete the survey for finite-source/point-lens (FSPL) giant-source events in 2016-2019 KMTNet microlensing data. The 30 FSPL events show a clear gap in Einstein radius, $9\,\mu{\rm as}<\theta_{\rm E} <26\,\mu{\rm as}$, which is consistent with the gap in Einstein timescales near $t_{\rm E}\sim 0.5\,$days found by Mroz et al. (2017) in an independent sample of point-source/point-lens (PSPL) events. We demonstrate that the two surveys are consistent. We estimate that the 4 events below this gap are due to a power-law distribution of free-floating planet candidates (FFPs) $dN_{\rm FFP}/d\log M = (0.4\pm 0.2)\,(M/38 M_\oplus)^{-p}$/star, with $0.9\lesssim p \lesssim 1.2$. There are substantially more FFPs than known bound planets, implying that the bound planet power-law index $\gamma=0.6$ is likely shaped by the ejection process at least as much as by formation. The mass density per decade of FFPs in the Solar neighborhood is of the same order as that of 'Oumuamua-like objects. In particular, if we assume that 'Oumuamua is part of the same process that ejected the FFPs to very wide or unbound orbits, the power-law index is $p=0.92\pm 0.06$. If the Solar System's endowment of Neptune-mass objects in Neptune-like orbits is typical, which is consistent with the results of Poleski et al. (2021), then these could account for a substantial fraction of the FFPs in the Neptune-mass range. 

</div>

<div id="div_fig1">

<img src="tmp_2204.03269/./cmdall.png" alt="Fig1.1" width="50%"/><img src="tmp_2204.03269/./lb.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** Color-magnitude locations of the 30 FSPL events relative to
the clump, shown against a background of field stars from the
CMD of KMT-2019-BLG-2555, i.e., the same as was used in Figure 8 of
\citet{kb192073}.  The events are color-coded by discovery year.
FFPs have red cores.  As expected, the sources are concentrated in
the clump (the highest concentration of giants), are somewhat
over-represented on the upper giant branch (due to larger cross sections),
and, conversely somewhat under-represented on the lower giant branch.
The largest outlier, OGLE-2017-BLG-0905 (green point near top), is discussed in
Appendix \ref{sec:individual}, while the second largest, KMT-2019-BLG-1143
 (blue point toward
the right), was discussed by \citet{kb192073}.  The lack of detections
with ordinate $\ga 15.5$ is discussed in Section \ref{sec:sample}, with
reference to \citet{kb192073}.
Field positions of 30 FSPL events in Galactic coordinates, color-coded
accorded to year of discovery (see labels in Figure \ref{fig:cmdall}).
The background black points are (as in Figure 4 of \citealt{kb192073})
EventFinder events from 2019.  As in that Figure, the KMT fields are
shown as black squares, with blue field numbers.
Their observational cadences can be obtained from Figure 12 of
\citet{eventfinder}.
 (*fig:cmdall*)

</div>
<div id="div_fig2">

<img src="tmp_2204.03269/./cum3.png" alt="Fig3" width="100%"/>

**Figure 3. -** Cumulative Distribution of $\log\theta_$\e$$ for
30 FSPL events with giant-star sources
that were found in four seasons of KMTNet data, 2016-2019.  The absence
of detections $\theta_$\e$\la 4 $\muas$$ is due to selection effects,
while the paucity for $\theta_$\e$\ga 300 $\muas$$ is due to a combination
of selection effects and
the lack of bulge stars $M$\ga$ 1 M_\odot$.
The remaining feature, the  factor $\sim 3$ "Einstein Desert",
$8.8 $\muas$ <\theta_$\e$< 26 $\muas$$, is the signature of two populations, i.e.,
BDs/stars with masses $0.02$\la$ M/M_\odot $\la$ 1$ to the right and
FFPs with much lower masses to the left.
The former are detected mainly in the bulge, while the latter are
likely to be about equally drawn from the bulge and the disk.
See Section \ref{sec:power-law}.
This Figure can be compared to Figure 10 from \citet{kb192073}.
 (*fig:cum3*)

</div>
<div id="div_fig3">

<img src="tmp_2204.03269/./z.png" alt="Fig2" width="100%"/>

**Figure 2. -** Cumulative distribution of normalized impact parameters
$z_0=u_0/\rho$.  The red line connects the first and last points,
while the green line connects the first and penultimate points.
The underlying distribution is rigorously uniform in $z_0$, so the
eye is struck by the absence of detections toward $z_0\simeq 1$.
According to a Kolmogorov-Smirnov (KS) test, this is not significant,
but the effect
may be real because the KS test does not capture all the physics.
See Section \ref{sec:sample}.  The distribution relative to the
green line suggests that the sample is homogeneously selected for
$z_0$\la$ 0.8$.
 (*fig:z*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\hl}$
$\newcommand{\cii}{C \textsc{ii}]}$
$\newcommand{\ciii}{C \textsc{iii}]}$
$\newcommand{\mgii}{Mg \textsc{ii}}$
$\newcommand{\civ}{C \textsc{iv}}$
$\newcommand{\feii}{Fe \textsc{ii}}$
$\newcommand{\nv}{N \textsc{v}}$
$\newcommand{\niv}{N \textsc{iv}]}$
$\newcommand{\siii}{Si \textsc{ii}}$
$\newcommand{\siiii}{Si \textsc{iii}}$
$\newcommand{\siiv}{Si \textsc{iv}}$
$\newcommand{\oiv}{O \textsc{iv}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\oi}{O \textsc{i}]}$
$\newcommand{\oii}{O \textsc{ii}]}$
$\newcommand{\oiii}{O \textsc{iii}]}$
$\newcommand{\alii}{Al \textsc{ii}}$
$\newcommand{\aliii}{Al \textsc{iii}}$
$\newcommand{\lya}{Ly \textsc{\alpha}}$
$\newcommand{\ciiil}{C \textsc{iii}] \lambda1909}$
$\newcommand{\mgiil}{Mg \textsc{ii} \lambda2799}$
$\newcommand{\civl}{C \textsc{iv} \lambda1549}$
$\newcommand{\nvl}{N \textsc{v} \lambda1240}$
$\newcommand{\nivl}{N \textsc{iv}] \lambda1486}$
$\newcommand{\siiil}{Si \textsc{ii} \lambda1263}$
$\newcommand{\siiiil}{Si \textsc{iii} \lambda1887}$
$\newcommand{\siivl}{Si \textsc{iv} \lambda1398}$
$\newcommand{\oivl}{O \textsc{iv} \lambda1402}$
$\newcommand{\heiil}{He \textsc{ii} \lambda1640}$
$\newcommand{\oiiil}{O \textsc{iii}] \lambda1663}$
$\newcommand{\aliil}{Al \textsc{ii} \lambda1671}$
$\newcommand{\aliiil}{Al \textsc{iii} \lambda1857}$
$\newcommand{\lyal}{Ly \textsc{\alpha} \lambda1216}$
$\newcommand{\nvciv}{\nv/\civ}$
$\newcommand{\siivoivciv}{(\siiv+\oiv)/\civ}$
$\newcommand{\oiiialiiciv}{(\oiii+\alii)/\civ}$
$\newcommand{\aliiiciv}{\aliii/\civ}$
$\newcommand{\siiiiciv}{\siiii/\civ}$
$\newcommand{\ciiiciv}{\ciii/\civ}$
$\newcommand{\angstrom}{\mbox{\normalfont\AA}}$
$\newcommand{\orcidChrisO}{\href{https://orcid.org/0000-0003-0017-349X}{\orcidicon}}$
$\newcommand{\orcidChrisW}{\href{https://orcid.org/0000-0002-4569-016X}{\orcidicon}}$
$\newcommand{\orcidSamuel}{\href{https://orcid.org/0000-0001-9372-4611}{\orcidicon}}$
$\newcommand{\orcidFuyan}{\href{https://orcid.org/0000-0002-1620-0897}{\orcidicon}}$
$\newcommand{\orcidChiara}{\href{https://orcid.org/0000-0002-5941-5214}{\orcidicon}}$
$\newcommand{\orcidJT}{\href{https://orcid.org/0000-0002-4544-8242}{\orcidicon}}$
$\newcommand{\orcidOnoue}{\href{https://orcid.org/0000-0003-2984-6803}{\orcidicon}}$
$\newcommand{\orcidValentina}{\href{https://orcid.org/0000-0003-3693-3091}{\orcidicon}}$
$\newcommand{\orcidManuela}{\href{https://orcid.org/0000-0002-4314-021X}{\orcidicon}}$
$\newcommand{\orcidjinyi}{\href{https://orcid.org/0000-0001-5287-4242}{\orcidicon}}$
$\newcommand{\orcidfanxiaohui}{\href{https://orcid.org/0000-0003-3310-0131}{\orcidicon}}$
$\newcommand{\orcidACE}{\href{https://orcid.org/0000-0003-2895-6218}{\orcidicon}}$
$\newcommand{\orcidFeige}{\href{https://orcid.org/0000-0002-7633-431X}{\orcidicon}}$
$\newcommand{\orcidEPF}{\href{https://orcid.org/0000-0002-6822-2254}{\orcidicon}}$
$\newcommand{\orcidSarahBosman}{\href{https://orcid.org/$
$0000-0001-8582-7012}{\orcidicon}}$
$\newcommand{\orcidYongda}{\href{https://orcid.org/$
$0000-0003-3307-7525}{\orcidicon}}$
$\newcommand{\orcidFabian}{\href{https://orcid.org/0000-0003-4793-7880}{\orcidicon}}$
$\newcommand{\orcidGeorgeB}{\href{https://orcid.org/0000-0003-2344-263X}{\orcidicon}}$
$\newcommand{\orcidGuido}{\href{https://orcid.org/0000-0002-6830-9093}{\orcidicon}}$
$\newcommand{\orcidEduardo}{\href{https://orcid.org/0000-0002-2931-7824}{\orcidicon}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}$
$\newcommand{$\cii$}{C $\textsc${ii}]}$
$\newcommand{$\cii$i}{C $\textsc${iii}]}$
$\newcommand{$\mgii$}{Mg $\textsc${ii}}$
$\newcommand{$\civ$}{C $\textsc${iv}}$
$\newcommand{$\feii$}{Fe $\textsc${ii}}$
$\newcommand{$\nv$}{N $\textsc${v}}$
$\newcommand{$\niv$}{N $\textsc${iv}]}$
$\newcommand{$\siii$}{Si $\textsc${ii}}$
$\newcommand{$\siii$i}{Si $\textsc${iii}}$
$\newcommand{$\siiv$}{Si $\textsc${iv}}$
$\newcommand{$\oiv$}{O $\textsc${iv}}$
$\newcommand{$\heii$}{He $\textsc${ii}}$
$\newcommand{$\oi$}{O $\textsc${i}]}$
$\newcommand{$\oi$i}{O $\textsc${ii}]}$
$\newcommand{$\oi$ii}{O $\textsc${iii}]}$
$\newcommand{$\alii$}{Al $\textsc${ii}}$
$\newcommand{$\alii$i}{Al $\textsc${iii}}$
$\newcommand{$\lya$}{Ly $\textsc${\alpha}}$
$\newcommand{$\cii$il}{C $\textsc${iii}] \lambda1909}$
$\newcommand{$\mgii$l}{Mg $\textsc${ii} \lambda2799}$
$\newcommand{$\civ$l}{C $\textsc${iv} \lambda1549}$
$\newcommand{$\nv$l}{N $\textsc${v} \lambda1240}$
$\newcommand{$\niv$l}{N $\textsc${iv}] \lambda1486}$
$\newcommand{$\siii$l}{Si $\textsc${ii} \lambda1263}$
$\newcommand{$\siii$il}{Si $\textsc${iii} \lambda1887}$
$\newcommand{$\siiv$l}{Si $\textsc${iv} \lambda1398}$
$\newcommand{$\oiv$l}{O $\textsc${iv} \lambda1402}$
$\newcommand{$\heii$l}{He $\textsc${ii} \lambda1640}$
$\newcommand{$\oi$iil}{O $\textsc${iii}] \lambda1663}$
$\newcommand{$\alii$l}{Al $\textsc${ii} \lambda1671}$
$\newcommand{$\alii$il}{Al $\textsc${iii} \lambda1857}$
$\newcommand{$\lya$l}{Ly $\textsc${\alpha} \lambda1216}$
$\newcommand{$\nv$civ}{$\nv$/$\civ$}$
$\newcommand{$\siiv$oivciv}{($\siiv$+$\oiv$)/$\civ$}$
$\newcommand{$\oi$iialiiciv}{($\oi$ii+$\alii$)/$\civ$}$
$\newcommand{$\alii$iciv}{$\alii$i/$\civ$}$
$\newcommand{$\siii$iciv}{$\siii$i/$\civ$}$
$\newcommand{$\cii$iciv}{$\cii$i/$\civ$}$
$\newcommand{$\angstrom$}{\mbox{\normalfont\AA}}$
$\newcommand{$\orcidChrisO$}{\href{https://orcid.org/0000-0003-0017-349X}{\orcidicon}}$
$\newcommand{$\orcidChrisW$}{\href{https://orcid.org/0000-0002-4569-016X}{\orcidicon}}$
$\newcommand{$\orcidSamuel$}{\href{https://orcid.org/0000-0001-9372-4611}{\orcidicon}}$
$\newcommand{$\orcidFuyan$}{\href{https://orcid.org/0000-0002-1620-0897}{\orcidicon}}$
$\newcommand{$\orcidChiara$}{\href{https://orcid.org/0000-0002-5941-5214}{\orcidicon}}$
$\newcommand{$\orcidJT$}{\href{https://orcid.org/0000-0002-4544-8242}{\orcidicon}}$
$\newcommand{$\orcidOnoue$}{\href{https://orcid.org/0000-0003-2984-6803}{\orcidicon}}$
$\newcommand{$\orcidValentina$}{\href{https://orcid.org/0000-0003-3693-3091}{\orcidicon}}$
$\newcommand{$\orcidManuela$}{\href{https://orcid.org/0000-0002-4314-021X}{\orcidicon}}$
$\newcommand{$\orcidjinyi$}{\href{https://orcid.org/0000-0001-5287-4242}{\orcidicon}}$
$\newcommand{$\orcidfanxiaohui$}{\href{https://orcid.org/0000-0003-3310-0131}{\orcidicon}}$
$\newcommand{$\orcidACE$}{\href{https://orcid.org/0000-0003-2895-6218}{\orcidicon}}$
$\newcommand{$\orcidFeige$}{\href{https://orcid.org/0000-0002-7633-431X}{\orcidicon}}$
$\newcommand{$\orcidEPF$}{\href{https://orcid.org/0000-0002-6822-2254}{\orcidicon}}$
$\newcommand{\orcidSarahBosman}{\href{https://orcid.org/$
$0000-0001-8582-7012}{\orcidicon}}$
$\newcommand{\orcidYongda}{\href{https://orcid.org/$
$0000-0003-3307-7525}{\orcidicon}}$
$\newcommand{$\orcidFabian$}{\href{https://orcid.org/0000-0003-4793-7880}{\orcidicon}}$
$\newcommand{$\orcidGeorgeB$}{\href{https://orcid.org/0000-0003-2344-263X}{\orcidicon}}$
$\newcommand{$\orcidGuido$}{\href{https://orcid.org/0000-0002-6830-9093}{\orcidicon}}$
$\newcommand{$\orcidEduardo$}{\href{https://orcid.org/0000-0002-2931-7824}{\orcidicon}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

#  Quasar Broad-Line Regions in the XQR-30 Sample

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2204.03335-b31b1b.svg)](https://arxiv.org/abs/2204.03335) _MNRAS accepted, 24 pages, 17 figures, 4 tables_

</div>
<div id="authors">

Samuel Lai, et al. -- incl., <mark><mark>Sarah E.I. Bosman</mark></mark>, <mark><mark>Jan-Torge Schindler</mark></mark>, <mark><mark>Fabian Walter</mark></mark>

</div>
<div id="abstract">

**Abstract:** The elemental abundances in the broad-line regions of high-redshift quasars trace the chemical evolution in the nuclear regions of massive galaxies in the early universe. In this work, we study metallicity-sensitive broad emission-line flux ratios in rest-frame UV spectra of 25 high-redshift (5.8 < z < 7.5) quasars observed with the VLT/X-shooter and Gemini/GNIRS instruments, ranging over$\log\left({\rm{M}_{\rm{BH}}/\rm{M}_{\odot}}\right)= 8.4-9.8$in black hole mass and$\log\left(\rm{L}_{\rm{bol}}/\rm{erg   s}^{-1}\right)= 46.7-47.7$in bolometric luminosity. We fit individual spectra and composites generated by binning across quasar properties: bolometric luminosity, black hole mass, and blueshift of the$\civ$line, finding no redshift evolution in the emission-line ratios by comparing our high-redshift quasars to lower-redshift (2.0 < z < 5.0) results presented in the literature. Using\texttt{Cloudy}-based locally optimally-emitting cloud photoionisation model relations between metallicity and emission-line flux ratios, we find the observable properties of the broad emission lines to be consistent with emission from gas clouds with metallicity that are at least 2-4 times solar. Our high-redshift measurements also confirm that the blueshift of the$\civ$emission line is correlated with its equivalent width, which influences line ratios normalised against$\civ$. When accounting for the$\civ$blueshift, we find that the rest-frame UV emission-line flux ratios do not correlate appreciably with the black hole mass or bolometric luminosity.

</div>

<div id="div_fig1">

<img src="tmp_2204.03335/./figs/line_ratio_blueshift.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\nv$civ and $\siiv$oivciv flux ratios as a function of the $\civ$ blueshift of quasars in the sample. Blueshift composites are presented in black while individual fits are mapped onto a blue-green gradient scaled to the black hole mass. The minimum $\civ$ blueshift error is 200 km s$^{-1}$ based on the 1\AA wavelength bins, but the contribution from the systemic redshift error is added in quadrature for an average total of 230 km s$^{-1}$ uncertainty. The Spearman correlation coefficients and p-values are derived from the fits to individual quasars. $\hl${The orange shaded space indicates a range of line ratios which are consistent with the metallicity indicated in the secondary axis based on photoionisation calculations with different ionizing SEDs}. The overlapping region in the $\nv$civ plot indicates a sub-space of parameters which is consistent with both $\rm{Z} = 10 \rm{Z}_{\odot}$ and $\rm{Z} = 20 \rm{Z}_{\odot}$. The line ratio correlation with the $\civ$ blueshift is more significant than the correlation with black hole mass or bolometric luminosity. (*fig:blueshift_line_ratio_results*)

</div>
<div id="div_fig2">

<img src="tmp_2204.03335/./figs/CIV_Blueshift_EW.png" alt="Fig1" width="100%"/>

**Figure 1. -** $\civ$ equivalent width as a function of $\civ$ blueshift. There is a moderate anti-correlation between these two quantities, implying that weaker $\civ$ lines are more strongly blueshifted and stronger lines are less blueshifted. The outlier with high EW and blueshift is J1216+4519. Its SNR is low ($\sim 7$ per pixel) which is reflected by the large error in EW. (*fig:CIV_blueshift_ew*)

</div>
<div id="div_fig3">

<img src="tmp_2204.03335/./figs/line_ratio_Lbol.png" alt="Fig9.1" width="50%"/><img src="tmp_2204.03335/./figs/line_ratio_Mbh.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** $\nv$civ and $\siiv$oivciv flux ratios as a function of the quasar bolometric luminosity (left) and virially estimated black hole mass (right). The low-redshift sample (2.0 < $z$ < 5.0) indicated in grey is from \citet{Xu2018} while another higher-redshift comparison sample indicated in red is sourced from \citet{Wang_2021}. Our sample is presented in blue and black. Square points with capped error bars indicate composites while circular points indicate individual fits. Not all individual quasars involved in the composites are plotted. The single red square denotes the composite from \citet{Wang_2021}. The black hole masses in this study and in \citet{Wang_2021} are estimated with single-epoch virial estimates using the $\mgii$ emission line, while the \citet{Xu2018} study uses the $\civ$ emission line. $\hl${The orange shaded space indicates a range of line ratios which are consistent with the metallicity indicated in the secondary axis based on photoionisation calculations with different ionizing SEDs}. The overlapping region in the $\nv$civ plot indicates a range of line ratios which is consistent with both $\rm{Z} = 10 \rm{Z}_{\odot}$ and $\rm{Z} = 20 \rm{Z}_{\odot}$\citep[e.g.][]{Hamann_2002, Nagao_2006}. Metallicity values larger than 10 Z$_{\odot}$ are extrapolated. (*fig:line_ratio_Lbol_Mbh*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('exports/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in sorted(res, key=lambda x:x[1])]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

18  publications in the last 7 days.
	 exports/2203.15811.md
	 exports/2203.15822.md
	 exports/2203.16504.md
	 exports/2203.16734.md
	 exports/2203.16735.md
	 exports/2203.16856.md
	 exports/2203.16959.md
	 exports/2204.00342.md
	 exports/2204.00793.md
	 exports/2204.01245.md
	 exports/2204.01758.md
	 exports/2204.01824.md
	 exports/2204.02017.md
	 exports/2204.02109.md
	 exports/2204.02998.md
	 exports/2204.03253.md
	 exports/2204.03269.md
	 exports/2204.03335.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("exports/index2.html", 'w') as fout:
    fout.write(page)

# Debugging papers

In [13]:
raise NotImplementedError("Manual Stop")

NotImplementedError: Manual Stop

In [ ]:
from IPython.display import display, Markdown
from TexSoup import TexSoup
import re

def bracket_error(source: str):
    """ Find problematic portions of the document """
    
    print("len(source)", len(source))
    
    # Checking header
    begin_doc = next(re.finditer(r'\\begin\{document\}', doc.source)).span()[1]
    header = source[:begin_doc]
    text = header + r"\n\end{document}"

    try:
        # print("Header check... ", end='')
        TexSoup(text)
        display(Markdown(f"**[OK]** - Header"))
    except:
        raise RuntimeError("Error in the header")
        
    # Check the text per section until the end.
    # Do not stop and try them all.
    
    problematic_text = []
    
    sections = ([(0, begin_doc, 'until first section')] + 
                [(g.span()[0], g.span()[1], g.group()) for g in re.finditer(r'\\section\{.*\}', source)] +
                [(g.span()[0], g.span()[1], g.group()) for g in re.finditer(r'\\begin\{appendix\}', source)]
               )
    sections.append([len(source), len(source), 'end'])
    
    sections = sorted(sections, key=lambda x: x[0])
    
    prev_pos, prev_name = (0, 'header')
    parsed = []
    
    for span, span_end, name in sections:

        if span - prev_pos <= 0:
            continue
            

        text = source[prev_pos:span]
        if prev_pos > begin_doc:
            text = r"\n\begin{document}" + text + r"\n\end{document}"
        else:
            text = text + r"\n\end{document}"
        try:
            #print(f"{prev_pos}:{prev_name}-->{span}:{name} check... ", end='')
            parsed.append(TexSoup(text, tolerance=1))  # allow not ending env
            display(Markdown(f"**[OK]** - *{prev_pos}*:{prev_name} &rarr; *{span}*:{name}"))
            # print("ok")

            prev_pos = span
            prev_name = name
        except:
            # print(f"error between {prev_pos} and {span}")
            display(Markdown(f"**[ERR]** *{prev_pos}*:{prev_name} &rarr; *{span}*:{name}"))
            problematic_text.append((prev_pos, source[prev_pos:span]))
            prev_pos = span
            prev_name = name
            # raise
    return problematic_text, parsed


def check_environment(text, offset=0):
    """ Check environment """
    env = re.compile(r"\\begin\{(?P<env>.*)\}(.*)\\end\{(?P=env)\}", re.DOTALL)

    for match in env.finditer(text):
        beg, end = match.span()
        beg += offset
        end += offset
        envname = match.groups()[0]
        try:
            latex.TexSoup(match.group())
        except Exception as e:
            display(e)
            print(f"Error in {envname:s} between {beg} and {end}")
            return match.groups()[1], beg, end

In [ ]:
import importlib
importlib.reload(latex)
which = "2204.03253"
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'

if not os.path.isdir(folder):
    folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')

try:
    doc = latex.LatexDocument(folder, validation=validation)    
except AffiliationError as affilerror:
    msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
    print(msg)


# Hack because sometimes author parsing does not work well
if (len(doc.authors) != len(paper['authors'])):
    doc._authors = paper['authors']
if (doc.abstract) in (None, ''):
    doc._abstract = paper['abstract']

doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
doc.highlight_authors_in_list(hl_list)

full_md = doc.generate_markdown_text()

In [ ]:
Markdown(full_md)

In [ ]:
#[check_environment(k) for k in bracket_error(doc.source)]
_, _, a = latex.get_content_per_section(doc.source, verbose=True)
if not a:
    print("no issues per section")
for ak in a:
    r = check_environment(ak[1], offset=ak[0])
    print(r[1], r[2])
    print(r[0])

In [14]:
[display(k) for k in candidates]


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02985-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02985) | **More Evidence for Variable Helium Absorption from HD 189733b**  |
|| <mark>Michael Zhang</mark>, et al. -- incl., <mark>Heather A. Knutson</mark>, <mark>Laura Kreidberg</mark> |
|*Comments*| *submitted to AJ*|
|**Abstract**| We present a new Keck/NIRSPEC observation of metastable helium absorption from the upper atmosphere of HD 189733b, a hot Jupiter orbiting a nearby moderately active star. We measure an average helium transit depth of $0.420 \pm 0.013$% integrated over the [-20, 20] km/s velocity range. Comparing this measurement to eight previously published transit observations with different instruments, we find that our depth is 32% (9$\sigma$) lower than the average of the three CARMENES transits, but only 16% (4.4$\sigma$) lower than the average of the five GIANO transits. We perform 1D hydrodynamical simulations of the outflow, and find that XUV variability on the order of 33%--common for this star--can change the helium absorption depth by 60%. We conclude that changes in stellar XUV flux can explain the observational variability in helium absorption. 3D models are necessary to explore other sources of variability, such as shear instability and changing stellar wind conditions. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02989-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02989) | **Birth of the Galactic Disk Revealed by the H3 Survey**  |
|| Charlie Conroy, et al. -- incl., <mark>Dennis Zaritsky</mark> |
|*Comments*| *13 pages, 10 figures, submitted to ApJ*|
|**Abstract**| We use chemistry ([alpha/Fe] and [Fe/H]), main sequence turnoff ages, and kinematics determined from H3 Survey spectroscopy and Gaia astrometry to identify the birth of the Galactic disk. We separate in-situ and accreted stars on the basis of angular momenta and eccentricities. The sequence of high-alpha in-situ stars persists down to at least [Fe/H]=-2.5 and shows unexpected non-monotonic behavior: with increasing metallicity the population first declines in [alpha/Fe], then increases over the range -1.3<[Fe/H]<-0.7, and then declines again at higher metallicities. The number of stars in the in-situ population rapidly increases above [Fe/H]=-1. The average kinematics of these stars are hot and independent of metallicity at [Fe/H]<-1 and then become increasingly cold and disk-like at higher metallicities. The ages of the in-situ, high-alpha stars are uniformly very old (13 Gyr) at [Fe/H]<-1.3, and span a wider range (8-12 Gyr) at higher metallicities. Interpreting the chemistry with a simple chemical evolution model suggests that the non-monotonic behavior is due to a significant increase in star formation efficiency, which began 13 Gyr ago. These results support a picture in which the first 1 Gyr of the Galaxy was characterized by a "simmering phase" in which the star formation efficiency was low and the kinematics had substantial disorder with some net rotation. The disk then underwent a dramatic transformation to a "boiling phase", in which the star formation efficiency increased substantially, the kinematics became disk-like, and the number of stars formed increased tenfold. We interpret this transformation as the birth of the Galactic disk at z~4. The physical origin of this transformation is unclear and does not seem to be reproduced in current galaxy formation models. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.02998-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.02998) | **High-contrast, high-angular resolution view of the GJ 367 exoplanet  system**  |
|| Wolfgang Brandner, Per Calissendorff, <mark>Neige Frankel</mark>, Faustine Cantalloube |
|*Comments*| *9 pages, 8 figures, accepted by MNRAS*|
|**Abstract**| We search for additional companions in the GJ 367 exoplanet system, and aim at better constraining its age and evolutionary status. We analyse high contrast direct imaging observations obtained with HST/NICMOS, VLT/NACO, and VLT/SPHERE. We investigate and critically discuss conflicting age indicators based on theoretical isochrones and models for Galactic dynamics. A comparison of GAIA EDR3 parallax and photometric measurements with theoretical isochrones suggest a young age $\le$60 Myr for GJ 367. The star's Galactic kinematics exclude membership to any nearby young moving group or stellar stream. Its highly eccentric Galactic orbit, however, is atypical for a young star. Age estimates considering Galactic dynamical evolution are most consistent with an age of 1 to 8 Gyr. We find no evidence for a significant mid-infrared excess in the WISE bands, suggesting the absence of warm dust in the GJ 367 system. The direct imaging data provide significantly improved detection limits compared to previous studies. At 530 mas (5 au) separation, the SPHERE data achieve a 5 sigma contrast of $2.6 \times 10^{-6}$. The data exclude the presence of a stellar companion at projected separations $\ge$0.4 au. At projected separations $\ge$5 au we can exclude substellar companions with a mass $\ge$ 1.5 M$_{\rm Jup}$ for an age of 50 Myr, and $\ge$ 20 M$_{\rm Jup}$ for an age of 5 Gyr. By applying the stellar parameters corresponding to the 50 Myr isochrone, we derive a bulk density of $\rho_{\rm planet} = 6.2$ g/cm$^3$ for GJ 367b, which is 25% smaller than a previous estimate. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03008-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03008) | **First Census of Gas-phase Metallicity Gradients of Star-forming Galaxies  in Overdense Environments at Cosmic Noon**  |
|| Zihao Li, et al. -- incl., <mark>Xian Zhong Zheng</mark>, <mark>Alaina L. Henry</mark> |
|*Comments*| *13 pages, 6 figures, 1 table. Accepted for publication in ApJL*|
|**Abstract**| We report the first spatially resolved measurements of gas-phase metallicity radial gradients in star-forming galaxies in overdense environments at $z\gtrsim2$. The spectroscopic data are acquired by the \mg\ survey, a Hubble Space Telescope (HST) cycle-28 medium program. This program is obtaining 45 orbits of WFC3/IR grism spectroscopy in the density peak regions of three massive galaxy protoclusters (BOSS 1244, BOSS 1542 and BOSS 1441) at $z=2-3$. Our sample in the BOSS 1244 field consists of 20 galaxies with stellar-mass ranging from $10^{9.0}$ to $10^{10.3}$ \Msun\ , star formation rate (SFR) from 10 to 240 \Msun\,yr$^{-1}$, and global gas-phase metallicity (\oh) from 8.2 to 8.6. At $1\sigma$ confidence level, 2/20 galaxies in our sample show positive (inverted) gradients -- the relative abundance of oxygen increasing with galactocentric radius, opposite the usual trend. Furthermore, 1/20 shows negative gradients and 17/20 are consistent with flat gradients. This high fraction of flat/inverted gradients is uncommon in simulations and previous observations conducted in blank fields at similar redshifts. To understand this, we investigate the correlations among various observed properties of our sample galaxies. We find an anticorrelation between metallicity gradient and global metallicity of our galaxies residing in extreme overdensities, and a marked deficiency of metallicity in our massive galaxies as compared to their coeval field counterparts. We conclude that the cold-mode gas accretion plays an active role in shaping the chemical evolution of galaxies in the protocluster environments, diluting their central chemical abundance, and flattening/inverting their metallicity gradients. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03020-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03020) | **Double Source Lensing Probing High Redshift Cosmology**  |
|| <mark>Divij Sharma</mark>, Eric V. Linder |
|*Comments*| *7 pages, 9 figures*|
|**Abstract**| Double source lensing, with two sources lensed by the same foreground galaxy, involves the distance between each source and the lens and hence is a probe of the universe away from the observer. The double source distance ratio also reduces sensitivity to the lens model and has good complementarity with standard distance probes. We show that using this technique at high redshifts $z>1$, to be enabled by data from the Euclid satellite and other surveys, can give insights on dark energy, both in terms of $w_0$-$w_a$ and redshift binned density. We find a dark energy figure of merit of 245 from combination of 256 double source systems with moderate quality cosmic microwave background and supernova data. Using instead five redshift bins between $z=1.1$-5, we could detect the dark energy density out to $z\approx5$, or make measurements ranging between 31$\sigma$ and 2.5$\sigma$ of its values in the bins. |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03358-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03358) | **Ultracool dwarfs identified using spectra in LAMOST DR7**  |
|| You-Fen Wang, et al. -- incl., <mark>Shuo Zhang</mark>, <mark>Zhong-Rui Bai</mark> |
|*Comments*| *13 pages, 11 figures, published in A&A, Volume 660. A38 (2022)*|
|**Abstract**| In this work, we identify 734 ultracool dwarfs with a spectral type of M6 or later, including one L0. Of this sample, 625 were studied spectroscopically for the first time. All of these ultracool dwarfs are within 360~pc, with a \textit{Gaia} G magnitude brighter than ~19.2 mag. By studying the spectra and checking their stellar parameters (Teff, logg, and [FeH] derived with the LAMOST pipeline, we found their cool red nature and their metallicity to be consistent with the nature of Galactic thin-disk objects. Furthermore, 77 of them show lithium absorption lines at 6708A, further indicating their young ages and substellar nature. Kinematics obtained through LAMOST radial velocities, along with the proper motion and parallax data from Gaia EDR3, also suggest that the majority of our targets are thin-disk objects. Kinematic ages were estimated through the relationship between the velocity dispersion and the average age for a certain population. Moreover, we identified 35 binaries, with 6 of them reported as binaries for the first time. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03423-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03423) | **The last three years: multiband gravitational-wave observations of  stellar-mass binary black holes**  |
|| <mark>Antoine Klein</mark>, et al. -- incl., <mark>Natalie Williams</mark> |
|*Comments*| *21 pages, 13 figures (including supplementary material)*|
|**Abstract**| Understanding the formation and evolution of the stellar-mass binary black holes discovered by LIGO and Virgo is a challenge that spans many areas of astrophysics, from stellar evolution, dynamics and accretion disks, to possible exotic early universe processes. Over the final years of their lives, stellar-mass binaries radiate gravitational waves that are first observable by space-based detectors (such as LISA) and then ground-based instruments (such as LIGO, Virgo and the next generation observatories Cosmic Explorer and the Einstein Telescope). Using state-of-the-art waveform models and parameter-estimation pipelines for both ground- and space-based observations, we show that (the expected handful of) these multiband observations will allow at least percent-level measurements of all 17 parameters that describe the binary, the possible identification of a likely host galaxy, and the forewarning of the merger days in advance allowing telescopes at multiple wavelengths to search for any electromagnetic signature associated to it. Multiband sources will therefore be a gold mine for astrophysics, but we also show that they are less useful as laboratories for fundamental tests of general relativity than has been previously suggested. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03457-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03457) | **Validation and interpretation of three-dimensional configuration of a  magnetic cloud flux rope**  |
|| Qiang Hu, et al. -- incl., <mark>Wen He</mark> |
|*Comments*| *Submitted to ApJ*|
|**Abstract**| One "strong" magnetic cloud (MC) with the magnetic field magnitude reaching $\sim$ 40 nT at 1 au during 2012 June 16-17 is examined in association with a pre-existing magnetic flux rope (MFR) identified on the Sun. The MC is characterized by a quasi-three dimensional (3D) flux rope model based on in situ measurements from the Wind spacecraft. The magnetic flux contents and other parameters are quantified. In addition, a correlative study with the corresponding measurements of the same structure crossed by the Venus Express (VEX) spacecraft at a heliocentric distance 0.7 au and with an angular separation $\sim 6^\circ$ in longitude is performed to validate the MC modeling results. The spatial variation between the Wind and VEX magnetic field measurements is attributed to the 3D configuration of the structure as featured by a knotted bundle of flux. The comparison of the magnetic flux contents between the MC and the source region on the Sun indicates that the 3D reconnection process accompanying an M1.9 flare may correspond to the magnetic reconnection between the field lines of the pre-existing MFR rooted in the opposite polarity footpoints. Such a process reduces the amount of the axial magnetic flux in the erupted flux rope, by approximately 50\%, in this case. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2204.03481-b31b1b.svg)](https://arxiv.org/abs/arXiv:2204.03481) | **Serendipitous discovery of radio flaring behaviour from a nearby M dwarf  with MeerKAT**  |
|| Alex Andersson, et al. -- incl., <mark>David Williams</mark> |
|*Comments*| *Accepted to MNRAS, 11 pages, 9 figures*|
|**Abstract**| We report on the detection of MKT J174641.0$-$321404, a new radio transient found in untargeted searches of wide-field MeerKAT radio images centred on the black hole X-ray binary H1743$-$322. MKT J174641.0$-$321404 is highly variable at 1.3 GHz and was detected three times during 11 observations of the field in late 2018, reaching a maximum flux density of 590 $\pm$ 60 $\mu$Jy. We associate this radio transient with a high proper motion, M dwarf star SCR~1746$-$3214 12 pc away from the Sun. Multiwavelength observations of this M dwarf indicate flaring activity across the electromagnetic spectrum, consistent with emission expected from dMe stars, and providing upper limits on quiescent brightness in both the radio and X-ray regimes. \textit{TESS} photometry reveals a rotational period for SCR~1746$-$3214 of $0.2292 \pm 0.0025$ days, which at its estimated radius makes the star a rapid rotator, comparable to other low mass systems. Dedicated spectroscopic follow up confirms the star as a mid-late spectral M dwarf with clear magnetic activity indicated by strong H$\alpha$ emission. This transient's serendipitous discovery by MeerKAT, along with multiwavelength characterisation, make it a prime demonstration of both the capabilities of the current generation of radio interferometers and the value of simultaneous observations by optical facilities such as MeerLICHT. Our results build upon the literature of of M dwarfs' flaring behaviour, particularly relevant to the habitability of their planetary systems. |

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]